In [ ]:
pip install amadeus

In [ ]:
"""
Example: list *all* flight offers that depart <origin> on <date>
using the Amadeus *test* environment.

Prerequisites
-------------
$ pip install amadeus python-dotenv           # or keep using plain `requests`
# create a .env file with:
AMADEUS_CLIENT_ID=your_test_key
AMADEUS_CLIENT_SECRET=your_test_secret
"""

import os
from datetime import datetime


In [ ]:

try:
    from amadeus import Client, ResponseError
except ImportError:
    Client = None        # fallback later if SDK isn't installed

#load_dotenv()            # pull credentials from .env or the environment

CLIENT_ID     = os.getenv("AMADEUS_CLIENT_ID")
CLIENT_SECRET = os.getenv("AMADEUS_CLIENT_SECRET")
HOSTNAME      = "test"   # <‑‑ keep “test” for the free sandbox

def sdk_get_flights(origin_iata: str, departure_date: str,
                    max_offers_per_dest: int = 20, adults: int = 1):
    """
    Return a flat list with every flight offer (price + itinerary segments)
    departing *origin_iata* on *departure_date*.
    """
    if Client is None:
        raise RuntimeError("Install the `amadeus` package to use the SDK path.")

    amadeus = Client(
        client_id     = CLIENT_ID,
        client_secret = CLIENT_SECRET,
        hostname      = HOSTNAME
    )

    try:
        # Step 1 – which destinations are served that day?
        dest_resp = amadeus.shopping.flight_destinations.get(
            origin        = origin_iata,
            departureDate = departure_date
        )
        destinations = [d["destination"] for d in dest_resp.data]

        all_offers = []
        # Step 2 – query each origin‑destination pair
        for dst in destinations:
            offers_resp = amadeus.shopping.flight_offers_search.get(
                originLocationCode      = origin_iata,
                destinationLocationCode = dst,
                departureDate           = departure_date,
                adults                  = adults,
                currencyCode            = "DKK",
                max                     = max_offers_per_dest
            )
            all_offers.extend(offers_resp.data)

        return all_offers

    except ResponseError as err:
        # The SDK already prints the HTTP details; re‑raise for visibility
        raise err


In [ ]:

if __name__ == "__main__":
    origin = "CPH"         # Madrid works reliably in the sandbox
    date   = datetime.now().strftime("%Y-%m-%d")

    print(f"\n=== Using SDK ===")
    if Client:
        flights = sdk_get_flights(origin, date)
        print(f"Total offers: {len(flights)}")
    else:
        print("SDK path skipped – install `amadeus` to enable it.")
